# 0.0 Imports

In [5]:
import re
import datetime
import inflection

import pandas           as pd
import numpy            as np
import seaborn          as sns
import umap.umap_       as umap
import matplotlib.cm    as cm

from plotly             import express          as px
from sklearn            import cluster          as ct
from sklearn            import metrics          as mt
from sklearn            import preprocessing    as pp
from matplotlib         import pyplot           as plt
from sklearn            import decomposition    as dd
from sklearn.manifold   import TSNE 
from sklearn            import ensemble         as en
from sklearn            import mixture          as mx
from sklearn.neighbors  import NearestNeighbors

from scipy.cluster      import hierarchy        as hc


## 0.2 Load Dataset

In [6]:
df_raw = pd.read_csv('../datasets/data.csv', encoding='ISO-8859-1')

# 1.0 Descrição dos Dados

In [8]:
df1 = df_raw.copy()

## 1.1 Rename Columns

In [9]:
cols_old = ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country']

snakecase = lambda x: inflection.underscore( x )

cols_new = list (map (snakecase, cols_old))

#
df1.columns = cols_new

## 1.2 Data dimensions

In [10]:
print ('Nmber of rows: {}'.format(df1.shape[0]))
print ('Nmber of columns: {}'.format(df1.shape[1]))

Nmber of rows: 541909
Nmber of columns: 8


## 1.3 Data Types

In [11]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
customer_id     float64
country          object
dtype: object

## 1.4 Check NA's

In [12]:
df1.isna().sum()

invoice_no           0
stock_code           0
description       1454
quantity             0
invoice_date         0
unit_price           0
customer_id     135080
country              0
dtype: int64

## 1.5 Replace NA

In [13]:
df_missing = df1.loc[df1['customer_id'].isna(), :]
df_not_missing = df1.loc[~df1['customer_id'].isna(), :]

In [14]:
# create reference

df_backup = pd.DataFrame(df_missing['invoice_no'].drop_duplicates())
df_backup['customer_id'] = np.arange(19000, 19000+len(df_backup), 1)
df_backup.head()

#merge original with reference df
df1 = pd.merge(df1, df_backup, on='invoice_no', how='left')

# coalesce
df1['customer_id'] = df1['customer_id_x'].combine_first(df1['customer_id_y'])
df1.sample(20)

# drop extra columns
df1 = df1.drop(columns=['customer_id_x', 'customer_id_y'], axis=1)

In [15]:
df1.isna().sum()

invoice_no         0
stock_code         0
description     1454
quantity           0
invoice_date       0
unit_price         0
country            0
customer_id        0
dtype: int64

In [16]:
# Remove na
df1 = df1.dropna( subset=['description'])
print('Removed data: {:.2f}'.format(1-(df1.shape[0] / df_raw.shape[0])))


Removed data: 0.00


In [17]:
df1.isna().sum()

invoice_no      0
stock_code      0
description     0
quantity        0
invoice_date    0
unit_price      0
country         0
customer_id     0
dtype: int64

## 1.6 Change dtypes

In [18]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
country          object
customer_id     float64
dtype: object

In [19]:
# invoice date
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'], format='%m/%d/%Y %H:%M', errors='coerce')
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'].dt.date)

df1['customer_id'] = df1['customer_id'].astype(int)

df1.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,United Kingdom,17850
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,United Kingdom,17850
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,United Kingdom,17850
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,United Kingdom,17850
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,United Kingdom,17850


## 1.7 Descriptive Statistics

In [20]:
num_attributes = df1.select_dtypes(include=['int64', 'float64'])
cat_attributes = df1.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]'])

### 1.7.1 Numerical Attributes

In [21]:
# central tendency - mean, median
ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

# dispersion std, min, max, range, skew, kurtosis
d1 = pd.DataFrame(num_attributes.apply(np.std)).T
d2 = pd.DataFrame(num_attributes.apply(np.min)).T
d3 = pd.DataFrame(num_attributes.apply(np.max)).T
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

# concatenate
m= pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
m

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,quantity,-80995.00,80995.0,161990.00,9.603129,3.00,218.007397,-0.281253,120252.449063
1,unit_price,-11062.06,38970.0,50032.06,4.623519,2.08,96.889538,186.257919,58848.017575
2,customer_id,12346.00,22709.0,10363.00,16678.153600,16241.00,2907.570262,0.493600,-0.792496


### 1.7.2 Categorical Atributes

In [22]:
cat_attributes.head()

,invoice_no,stock_code,description,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,United Kingdom
1,536365,71053,WHITE METAL LANTERN,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,United Kingdom


In [23]:
# Problema: Invoice a priori deveria ser apenas numeros

# invoice number
df_letter_invoices = df1.loc[df1['invoice_no'].apply(lambda x: bool(re.search('[^0-9+]', x))), :]
df_letter_invoices.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
141,C536379,D,Discount,-1,2010-12-01,27.50,United Kingdom,14527
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01,4.65,United Kingdom,15311
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01,1.65,United Kingdom,17548
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01,0.29,United Kingdom,17548
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01,0.29,United Kingdom,17548


In [24]:
#Explorando as linhas com invoices com letras
#Logo, será assumido que os invoices com letras se tratam de alguma forma de devolução/cancelamento
print('Quantidade de invoices que possui letras: {}'.format(len (df_letter_invoices)))
print('Quantidade de invoices que possui letras e com quantity menor que 0: {}'.format(len(df_letter_invoices[df_letter_invoices['quantity'] < 0])))

Quantidade de invoices que possui letras: 9291
Quantidade de invoices que possui letras e com quantity menor que 0: 9288


In [25]:
# stock code

#Check stock code with characters
df1.loc[df1['stock_code'].apply(lambda x: bool(re.search('^[a-zA-Z]+$', x))), 'stock_code'].unique()

# Ação:
# Remover stock_code in ['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY','DCGSSGIRL', 'PADS', 'B', 'CRUK']

array(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY',
       'DCGSSGIRL', 'PADS', 'B', 'CRUK'], dtype=object)

In [26]:
# country

len(df1['country'].unique())

38

In [27]:
df1['country'].value_counts(normalize = True)

country
United Kingdom          0.914089
Germany                 0.017569
France                  0.015833
EIRE                    0.015165
Spain                   0.004687
Netherlands             0.004387
Belgium                 0.003828
Switzerland             0.003704
Portugal                0.002811
Australia               0.002330
Norway                  0.002009
Italy                   0.001486
Channel Islands         0.001403
Finland                 0.001286
Cyprus                  0.001151
Sweden                  0.000855
Unspecified             0.000825
Austria                 0.000742
Denmark                 0.000720
Japan                   0.000662
Poland                  0.000631
Israel                  0.000550
USA                     0.000538
Hong Kong               0.000533
Singapore               0.000424
Iceland                 0.000337
Canada                  0.000279
Greece                  0.000270
Malta                   0.000235
United Arab Emirates    0.000126
Eu

In [28]:
df1[['customer_id', 'country']].drop_duplicates().groupby('country').count().reset_index().sort_values('customer_id', ascending=False)

,country,customer_id
36,United Kingdom,6133
14,Germany,95
13,France,90
10,EIRE,44
31,Spain,31
3,Belgium,25
33,Switzerland,24
27,Portugal,20
19,Italy,15
16,Hong Kong,15


# 2.0 Filtragem de Variáveis

In [29]:
df2 = df1.copy()

In [30]:
# ----- Numerical Attributes ----
# unit prince > 0.0
df2 = df2.loc[df2['unit_price'] >= 0.04, :]

# --------- Categorical Attributes -------
# stock code != ['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B', 'CRUK']
df2 = df2[~df2['stock_code'].isin(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B', 'CRUK'])]


#description
df2 = df2.drop( columns='description', axis=1)

#map
df2 = df2[~df2['country'].isin( ['European Community', 'Unspecified'])]

# bad users
df2 = df2[~df2['customer_id'].isin([16446])]


# ----- Numerical Attributes ----
# quantity
df2_returns = df2.loc[df2['quantity'] < 0, :]
df2_purchase = df2.loc[df2['quantity'] >= 0, :]



# 3.0 Feature Engineering

In [31]:
df3 = df2.copy()

## 3.1 Feature Creation

In [32]:
# data reference 

df_ref = df3.drop( ['invoice_no', 'stock_code', 'quantity',
                    'invoice_date', 'unit_price', 'country'] , axis=1).drop_duplicates(ignore_index=True)

### 3.1.1 Gross Revenue

In [33]:
# Gross Revenue (Faturamento) quantity * price = Faturamente

df2_purchase.loc[:, 'gross_revenue'] = df2_purchase.loc[:, 'quantity'] * df2_purchase.loc[:, 'unit_price']

# Monetary 

df_monetary = df2_purchase[['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index()
df_ref = pd.merge(df_ref, df_monetary, on='customer_id', how='left')
df_ref.isna().sum()


C:\Users\Ian\AppData\Local\Temp\ipykernel_4708\332750283.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_purchase.loc[:, 'gross_revenue'] = df2_purchase.loc[:, 'quantity'] * df2_purchase.loc[:, 'unit_price']


customer_id       0
gross_revenue    91
dtype: int64

### 3.1.2 Recency (Day from last purchase)

In [34]:
# Recency - Last day purchase
df_recency = df2_purchase[['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
df_recency['recency_days'] = (df2_purchase['invoice_date'].max() - df_recency['invoice_date']).dt.days
df_recency = df_recency[['customer_id', 'recency_days']].copy()
df_ref = pd.merge(df_ref, df_recency, on='customer_id', how='left')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
dtype: int64

### 3.1.5 Quantity of products purchased

In [35]:
# Quantidade de Produtos
df_freq = (df2_purchase[['customer_id', 'stock_code']].groupby('customer_id')
                                                    .count()
                                                    .reset_index()
                                                    .rename(columns={'stock_code': 'qtd_products'}))
df_ref = pd.merge(df_ref, df_freq, on='customer_id', how='left')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qtd_products     91
dtype: int64

### 3.1.5 Avg Recency Days

In [36]:
# Avg recency Days
df_aux = df2[['customer_id', 'invoice_date']].drop_duplicates().sort_values(['customer_id', 'invoice_date'], ascending=[False,False])
df_aux['next_customer_id'] = df_aux['customer_id'].shift() #next customer
df_aux['previous_invoice_date'] = df_aux['invoice_date'].shift() #next invoice date

df_aux['avg_recency_days'] = df_aux.apply(lambda x: ((x['invoice_date'] - x['previous_invoice_date']).days)*-1 if x['customer_id'] == x['next_customer_id'] else np.nan, axis=1)


df_aux = df_aux.drop(['invoice_date', 'next_customer_id', 'previous_invoice_date'], axis=1).dropna()

# average recency 
df_avg_recency_days = df_aux.groupby('customer_id').mean().reset_index()

# merge
df_ref = pd.merge(df_ref, df_avg_recency_days, on='customer_id', how='left')
df_ref.isna().sum()

customer_id            0
gross_revenue         91
recency_days          91
qtd_products          91
avg_recency_days    2816
dtype: int64

### 3.1.7 Frequency Purchase

In [37]:
df_aux = (df2_purchase[['customer_id', 'invoice_no', 'invoice_date']].drop_duplicates()
                                                   .groupby('customer_id')
                                                   .agg ( max_ = ('invoice_date', 'max'),
                                                          min_ = ('invoice_date', 'min'),
                                                          days_ = ('invoice_date', lambda x: ((x.max() - x.min()).days) + 1),
                                                          buy_ = ('invoice_no', 'count')                                                      
                                                   )).reset_index()

# Frequency
df_aux['frequency'] = df_aux[['buy_', 'days_']].apply( lambda x: x['buy_']/x['days_'] if x['days_'] != 0 else 0, axis=1)

# Merge
df_ref = pd.merge(df_ref, df_aux[['customer_id', 'frequency']], on='customer_id', how='left')

df_ref.isna().sum()


customer_id            0
gross_revenue         91
recency_days          91
qtd_products          91
avg_recency_days    2816
frequency             91
dtype: int64

### 3.1.8 Number of Returns

In [38]:
# Number of returns
df_returns = df2_returns[['customer_id', 'quantity']].groupby('customer_id').sum().reset_index().rename(columns={'quantity': 'qtd_returns'})
df_returns['qtd_returns'] = df_returns['qtd_returns'] * -1

df_ref = pd.merge(df_ref, df_returns, how='left', on='customer_id')
df_ref.loc[df_ref['qtd_returns'].isna(), 'qtd_returns'] = 0
df_ref.isna().sum()

customer_id            0
gross_revenue         91
recency_days          91
qtd_products          91
avg_recency_days    2816
frequency             91
qtd_returns            0
dtype: int64

# 4.0 EDA (Exploratory Data Analysis)

In [39]:
df4 = df_ref.dropna().copy()

In [40]:
df4.shape

(2968, 7)

## 4.3 Estudo do Espaço

In [41]:
# selected dataset
cols_select = ['customer_id', 'gross_revenue', 'recency_days', 'qtd_products', 'frequency', 'qtd_returns']
df43 = df4[cols_select].copy()

In [42]:
# selected dataset

mm = pp.MinMaxScaler()
df43['gross_revenue']           = mm.fit_transform(df43[['gross_revenue']])
df43['recency_days']            = mm.fit_transform(df43[['recency_days']])
df43['qtd_products']            = mm.fit_transform(df43[['qtd_products']])
df43['frequency']               = mm.fit_transform(df43[['frequency']])
df43['qtd_returns']             = mm.fit_transform(df43[['qtd_returns']])


### 4.3.4 Tree-Based Embedding


In [43]:
# training dataset

X = df4.drop(columns=['customer_id', 'gross_revenue'], axis=1)
y = df4['gross_revenue']

# model definition
rf_model = en.RandomForestRegressor(n_estimators=100, random_state=42)

# model training

rf_model.fit(X, y)

# Leaf

df_leaf = pd.DataFrame(rf_model.apply(X))

# dataframe Leaf

In [44]:
df_leaf.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,3466,2840,3723,3671,3655,3740,3583,3686,3664,3487,...,3404,3391,3374,3680,3575,3706,3664,3412,3626,3653
1,3294,2744,3686,3070,3024,2397,2631,3532,2765,3443,...,2436,3209,2304,2210,3388,2353,2773,3307,2574,3510
2,3487,2383,3695,3581,3562,3666,2762,3568,2733,3086,...,3369,3254,3493,3532,3505,3590,2597,3375,3531,3584
3,1227,546,1092,278,863,752,525,1093,1852,1622,...,972,1587,475,635,1615,1876,572,300,920,894
4,302,631,294,894,911,410,716,360,466,485,...,540,604,877,532,764,391,676,15,1316,26


In [45]:
# Reduzir a dimensionalidade
reducer = umap.UMAP(random_state=42)
embedding = reducer.fit_transform(df_leaf)

# embedding
df_tree = pd.DataFrame()
df_tree['embedding_x'] = embedding[:,0]
df_tree['embedding_y'] = embedding[:,1]


d:\Comunidade DS\Portifólio de Projetos\repos\ds_fidelidade\.venv\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
d:\Comunidade DS\Portifólio de Projetos\repos\ds_fidelidade\.venv\lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


# 7.0 Hyperparameter Fine-Tuning

In [46]:
X = df_tree.copy()

# 8.0 Model Training

In [47]:
df8 = X.copy()

## 8.2 GMM

In [48]:
# definition
k = 14
model = mx.GaussianMixture(n_components=k, random_state=32, n_init=500)

# training

labels = model.fit_predict(X)


### 8.1.1 Cluster Validation

In [49]:
## SS (Silhouette Score)
print('Silhouette Score: {}'.format(mt.silhouette_score(X, labels, metric='euclidean')))

Silhouette Score: 0.6410145163536072


# 9.0 Cluster Analysis

## 9.2 Cluster Profile

In [50]:
df92 = df4[cols_select].copy()
df92['cluster'] = labels

# change dtypes
df92['recency_days'] = df92['recency_days'].astype(int)
df92['qtd_products'] = df92['qtd_products'].astype(int)
df92['qtd_returns'] = df92['qtd_returns'].astype(int)

df92.head()

,customer_id,gross_revenue,recency_days,qtd_products,frequency,qtd_returns,cluster
0,17850,5391.21,372,297,17.000000,40,3
1,13047,3232.59,56,171,0.028302,35,9
2,12583,6705.38,2,232,0.040323,50,3
3,13748,948.25,95,28,0.017921,0,0
4,15100,876.00,333,3,0.073171,22,2


In [51]:
# Number of customer
df_cluster = df92[['customer_id', 'cluster']].groupby('cluster').count().reset_index()
df_cluster['perc_customer'] = 100*(df_cluster['customer_id'] / df_cluster['customer_id'].sum())

# Avg gross revenue
df_avg_gross_revenue = df92[['gross_revenue','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_gross_revenue, how='inner', on='cluster')

# Avg recency days
df_avg_recency_days = df92[['recency_days','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_recency_days, how='inner', on='cluster')

# Qtd products
df_qtd_products_no = df92[['qtd_products','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_qtd_products_no, how='inner', on='cluster')

# Frequency
df_frequency = df92[['frequency','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_frequency, how='inner', on='cluster')

# Qtd returns
df_qtd_returns = df92[['qtd_returns','cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_qtd_returns, how='inner', on='cluster')

df_cluster.sort_values(by='gross_revenue', ascending=False)

,cluster,customer_id,perc_customer,gross_revenue,recency_days,qtd_products,frequency,qtd_returns
3,3,468,15.768194,8836.136389,21.438034,424.134615,0.094571,149.271368
9,9,83,2.796496,3989.907590,24.168675,172.710843,0.090889,29.915663
11,11,149,5.020216,3183.995436,42.234899,128.778523,0.074251,19.382550
6,6,378,12.735849,2789.420132,51.375661,99.925926,0.054398,15.486772
13,13,62,2.088949,2058.942097,48.161290,175.435484,0.019621,14.870968
4,4,203,6.839623,1845.146847,45.054187,128.827586,0.019333,15.738916
1,1,142,4.784367,1625.483239,59.950704,75.056338,0.056054,35.612676
5,5,416,14.016173,1220.426154,61.324519,53.204327,0.042094,9.105769
8,8,278,9.366577,954.737698,77.489209,36.514388,0.081885,5.723022
12,12,22,0.741240,917.275909,58.409091,69.227273,0.012916,1.772727


# 11.0 Deploy To Production

In [54]:
df92.dtypes

customer_id        int64
gross_revenue    float64
recency_days       int64
qtd_products       int64
frequency        float64
qtd_returns        int64
cluster            int64
dtype: object

## 11.1 Insert into SQLite

In [55]:
import sqlite3
from sqlalchemy import create_engine

In [ ]:
# # create table
# query_create_table_insiders = '''
#     CREATE TABLE insiders (
#         customer_id     INTEGER,
#         gross_revenue   REAL,
#         recency_days    INTEGER,
#         qtd_products    INTEGER,
#         frequency       REAL,
#         qtd_returns     INTEGER,
#         cluster         INTEGER
#         )
# '''
# conn = sqlite3.connect('insiders_db.sqlite')
# conn.execute(query_create_table_insiders)
# conn.commit()
# conn.close()

# insert data
conn = create_engine('sqlite:///insiders_db.sqlite')
# df92.to_sql('insiders', con=conn, if_exists='append', index=False)

2968

In [57]:
# consulting database
query = '''
    SELECT * FROM insiders
    '''

df = pd.read_sql_query (query, conn)

In [58]:
df.shape

(2968, 7)

In [98]:
conn.close()